In [1]:
# Import statements
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.tsa.stattools import acf

In [ ]:
# Add in the NHS populations for normalisation
npis = pd.read_csv('England_LTLA_NHSER.csv')
npis = npis[['NHSER_name','LTLA_name']]
npis.rename(columns={'NHSER_name':'region','LTLA_name':'ltla20nm'},inplace=True)
npis.drop_duplicates(inplace=True)
pop = pd.read_csv('census_estimate.csv')
manual_dat = pd.DataFrame({'ltla20nm': ['Wycombe','South Bucks','Chiltern','Aylesbury Vale','Cornwall and Isles of Scilly', 'Hackney and City of London'], \
                           'region': ['South East', 'South East', 'South East', 'South East', 'South West', 'London']})
npis = pd.concat([npis,manual_dat])
ltla_pop = pd.read_csv('newCasesPCROnlyBySpecimenDate_ltla_2020.csv')
ltla_pop = ltla_pop[['area_code','area_name']]
ltla_pop.drop_duplicates(inplace=True)
ltla_pop = pd.merge(ltla_pop, npis, left_on='area_name', right_on='ltla20nm').drop(columns='ltla20nm')
ltla_pop = pd.merge(ltla_pop, pop)
nhs_pop = ltla_pop.groupby('region').sum().drop(columns=['area_code','area_name'])
nhs_pop.reset_index(inplace=True)
nhs_pop.to_csv('region_pop.csv')

In [66]:
# Preprocessing case data
cases21 = pd.read_csv('newCasesPCROnlyBySpecimenDate_ltla_2021.csv')
cases21 = cases21[['date','area_code','area_name','value']]
cases20 = pd.read_csv('newCasesPCROnlyBySpecimenDate_ltla_2020.csv')
cases20 = cases20[['date','area_code','area_name','value']]
cases20_additional = pd.read_csv('ltla_2023-12-14_agg.csv')
cases20_additional.rename(columns={'newCasesBySpecimenDate':'value','areaCode':'area_code','areaName':'area_name'},inplace=True)
cases20_additional.drop(columns=['areaType','cumCasesBySpecimenDate','cumCasesBySpecimenDateRate'],inplace=True)
cases20_additional = cases20_additional[cases20_additional['area_name'].isin(cases20['area_name'].values)]
cases20_additional['date'] = pd.to_datetime(cases20_additional['date'])
cases20_additional = cases20_additional[cases20_additional['date'] < '2020-11-01']
cases20_additional = cases20_additional[cases20_additional['date'] >= '2020-10-01']
cases20 = pd.concat([cases20,cases20_additional]).sort_index()
# Rename columns and take weekly incidence instead of daily
cases = pd.concat([cases20,cases21])
cases['date'] = pd.to_datetime(cases['date'])
cases.set_index('date', inplace=True)
cases.sort_index(inplace=True)

# Add in the NHS populations for normalisation
npis = pd.read_csv('England_LTLA_NHSER.csv')
npis = npis[['NHSER_name','LTLA_name']]
npis.rename(columns={'NHSER_name':'region','LTLA_name':'ltla20nm'},inplace=True)
npis.drop_duplicates(inplace=True)
manual_dat = pd.DataFrame({'ltla20nm': ['Wycombe','South Bucks','Chiltern','Aylesbury Vale','Cornwall and Isles of Scilly', 'Hackney and City of London'], \
                           'region': ['South East', 'South East', 'South East', 'South East', 'South West', 'London']})
npis = pd.concat([npis,manual_dat])
weekly_counts_total = pd.merge(cases.reset_index(),npis, left_on='area_name', right_on='ltla20nm').drop(columns=['ltla20nm','area_name','area_code'])
weekly_counts_total = weekly_counts_total.groupby(['date','region']).sum().reset_index()
pop = pd.read_csv('region_pop.csv')
# weekly_counts_total = pd.merge(weekly_counts_total,pop).drop(columns='Unnamed: 0')
# weekly_counts_total['incidence'] = weekly_counts_total['value'] / weekly_counts_total['population']

# national_mean_total = weekly_counts_total.groupby(['date'])['incidence'].mean().reset_index()
# national_mean_total.rename(columns={'incidence': 'national_mean_incidence'}, inplace=True)
# weekly_counts_total = pd.merge(weekly_counts_total, national_mean_total, on='date')
# weekly_counts_total['detrended'] = weekly_counts_total['incidence'] - weekly_counts_total['national_mean_incidence']
# # Convert to wide format for ease of future calculations
# wide_data_hosp_inc = weekly_counts_total.pivot(index='date', columns='region', values='detrended')

In [67]:
weekly_counts_total

,date,region,value
0,2020-06-01,East of England,142.0
1,2020-06-01,London,50.0
2,2020-06-01,Midlands,332.0
3,2020-06-01,North East and Yorkshire,273.0
4,2020-06-01,North West,262.0
...,...,...,...
4048,2021-12-31,Midlands,20460.0
4049,2021-12-31,North East and Yorkshire,20634.0
4050,2021-12-31,North West,19527.0
4051,2021-12-31,South East,13408.0
